# Manipulating an ENDF file

In addition to functions to navigate and traverse an ENDF tree, the `ENDFtk.tree` components also provide the necessary functionality to insert, remove or replace pieces of the ENDF file. Often, an evaluator will want to take data from one ENDF tree and add it to another ENDF tree, or even replace data in an ENDF tree with entirely new data.

The `ENDFtk.tree.Tape`, `ENDFtk.tree.Material` and `ENDFtk.tree.File` components all provide an `remove(...)` function that allows a user to remove specific pieces of the ENDF tree. `ENDFtk.tree.File` objects in an `ENDFtk.tree.Material` object and `ENDFtk.tree.Section` objects in an `ENDFtk.tree.File`object are supposed to be unique, so the `remove(...)` function will the remove the particular component if it is present.

This is slightly different for `ENDFtk.tree.Material` objects in an `ENDFtk.tree.Tape` object. `ENDFtk.tree.Material` objects with the same material number can actually be present multiple times in an `ENDFtk.tree.Tape` object. This is a rare occurence, and only used for processed ENDF tapes in which the same material can appear at multiple temperatures (this would happen when a user processes the same material at multiple temperatures in a processing code). In this case, the `remove(...)` function will remove any `ENDFtk.tree.Material` that has the given material number from the tree.

For example:

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

tape.material( 9437 ).file( 3 ).remove( 18 ) # remove only MF3 MT18
tape.material( 9437 ).remove( 3 )            # remove only MF3
tape.remove( 9437 )                          # remove the material in its entirety

The `ENDFtk.tree.Material` and `ENDFtk.tree.File` components all provide an `insert(...)` and `insert_or_replace(...)` function, respectively to insert a `ENDFtk.tree.File` and `ENDFtk.tree.Section` (or their parsed equivalent). The difference between the two functions is that the `insert(...)` function will only insert the component if the component was not already present (this will throw an exception in the process), while `insert_or_replace(...)` will remove the existing component (if it is present) and then insert the new one. The `insert(...)` function therefore provides some safety while `insert_or_replace(...)` does not provide any.

As with the `remove(...)` function, the behaviour for `ENDFtk.tree.Tape` is a special case. This component provides an `insert(...)` and `replace(...)` function. In this case, the `insert(...)` function inserts the material in the ENDF tape tree. If one or more materials with the same material number are already present, the new material is inserted after the materials that are already there. The `replace(...)` function on the other hand also inserts the material in the ENDF tape tree, but if one or more materials are already present with the same material number, the old materials are removed before inserting the new material.

For example:

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

section = tape.material( 9437 ).file( 3 ).section( 18 )
tape.material( 9437 ).file( 3 ).insert( section )            # this will throw an exception
tape.material( 9437 ).file( 3 ).insert_or_replace( section ) # this will not throw an exception

file = tape.material( 9437 ).file( 3 )
tape.material( 9437 ).insert( file )                         # this will throw an exception
tape.material( 9437 ).insert_or_replace( file )              # this will not throw an exception

material = tape.material( 9437 )
tape.insert( material )                                      # this will not throw an exception
tape.replace( material )                                     # this will not throw an exception

As mentioned above, these methods also function on the equivalent parsed `ENDFtk` components as shown here:

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

section = tape.material( 9437 ).file( 3 ).section( 18 ).parse()
tape.material( 9437 ).file( 3 ).insert( section )            # this will throw an exception
tape.material( 9437 ).file( 3 ).insert_or_replace( section ) # this will not throw an exception

file = tape.material( 9437 ).file( 3 ).parse()
tape.material( 9437 ).insert( file )                         # this will throw an exception
tape.material( 9437 ).insert_or_replace( file )              # this will not throw an exception

material = tape.material( 9437 ).parse()
tape.insert( material )                                      # this will not throw an exception
tape.replace( material )                                     # this will not throw an exception

The MF1 MT451 section in each material provides a directory or index of the content of the entire ENDF material. This directory must be kept up to date when inserting, replacing and/or removing pieces of the ENDF tree since this will modify the content of the tree. While we could have updated the directory each time the tree is modified, we decided against this behaviour since this would lead to multiple `parse()` operations (one for each time the tree is modified) - which is highly inefficient. `ENDFtk.tree.Tape` and `ENDFtk.tree.Material` components both have an `update_directory()` function defined on them to perform the update of the directory.

`ENDFtk.tree.File` and `ENDFtk.tree.Section` do not provide an `update_directory()` function since the ENDF directory is specifically defined as part of MF1 MT451.

For example:

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

tape.update_directory()
tape.material( 9437 ).update_directory()

## Application 1: inserting the reconstructed cross section data in the evaluated file
With the functionality presented above, we can now develop a simple tool that takes some of the content of a processed ENDF file coming out of the NJOY nuclear data processing tool and inserts it into the original evaluation, while removing other pieces to create a brand new ENDF file.

In [ ]:
import ENDFtk

evaluation = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )
reconstructed = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239-reconstructed.endf' )

resonances = reconstructed.material( 9437 ).file( 2 )
xs = reconstructed.material( 9437 ).file( 3 )

resonances.remove( 152 )

evaluation.material( 9437 ).insert_or_replace( resonances )
evaluation.material( 9437 ).insert_or_replace( xs )
evaluation.material( 9437 ).remove( 31 )
evaluation.material( 9437 ).remove( 32 )
evaluation.material( 9437 ).remove( 33 )
evaluation.material( 9437 ).remove( 34 )
evaluation.material( 9437 ).remove( 35 )
evaluation.update_directory()

evaluation.to_file( 'resources/n-Pu239-merged.endf' )

Please note that in the above example, we neglected to change a few flags in the MF1 MT451 section (in particular the `LRP` flag). While we could add this in the above example, it would detract from what we are trying to show: the simplicity of inserting, replacing and modifying an ENDF file.